In [1]:
import numpy as np
from astropy.io import fits
import matplotlib
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import os
from ProjectF import MLAData,classification, Object,storing

In [2]:
## Loading in data
PlateDir = os.path.normpath("D:\Data\Plate_Name.txt")
with open(PlateDir) as f:
    Spectra_Files = f.read().splitlines()

In [3]:
PLATEIDs = []
BinInfos = []
Flux = []
MJDs = []
TrainingDir = os.path.normpath("D:\Data")
TrainingFolder =  os.path.normpath("\Training")
slash =  os.path.normpath("\\")
for spectrum in Spectra_Files:
    plate_ = fits.open( TrainingDir +TrainingFolder+slash+ spectrum ,memmap=True)
    Bin_info_ = plate_[5].data
    Flux_ = plate_[0].data
    primhdu_ = plate_[0]
    PLATEIDs.append(primhdu_.header['PLATEID'])
    MJDs.append(primhdu_.header['MJD'])
    BinInfos.append(Bin_info_)
    Flux.append(Flux_)
    

In [4]:
list = fits.open(TrainingDir+slash+'Superset_DR12Q.fits',memmap=True)#opening file

supers=list[1].data # storing  BINTABLE extension data

In [5]:
Full_Data = storing(PLATEIDs,supers)

In [6]:
X,Y,Train_z, Train_mag = MLAData(Full_Data,BinInfos,Flux)

In [7]:
TestPlateDir = os.path.normpath("D:\Data\TestPlate_Name.txt")
with open(TestPlateDir) as f:
    Spectra_TestFiles = f.read().splitlines()
Spectra_TestFiles

['spPlate-4791-55889.fits',
 'spPlate-6970-56444.fits',
 'spPlate-5852-56034.fits']

In [8]:
TrainingDir = os.path.normpath("D:\Data")
TestingFolder =  os.path.normpath("\Test")
PLATEIDs_test = []
BinInfos_test = []
Flux_test = []
z=0
for spectrum_test in Spectra_TestFiles:
    plate_test = fits.open(spectrum_test ,memmap=True)
    Bin_info_test = plate_test[5].data
    Flux_t = plate_test[0].data
    primhdu_test = plate_test[0]
    PLATEIDs_test.append(primhdu_test.header['PLATEID'])
    BinInfos_test.append(Bin_info_test)
    Flux_test.append(Flux_t)


In [9]:
Full_Data_test = storing(PLATEIDs_test,supers)

In [10]:

X_test,Y_test,All_redshifts,All_Mag = MLAData(Full_Data_test,BinInfos_test,Flux_test)

In [11]:
hiddenlayer_format = (13)
backprop_method = 'lbfgs'
lr=0.00001
act = 'tanh'
mlp = MLPClassifier(hidden_layer_sizes=hiddenlayer_format,max_iter=500, solver = 'lbfgs',learning_rate_init=lr,activation=act) ##Think About

In [12]:
scaler = StandardScaler()  
scaler.fit(X)  
X = scaler.transform(X)  
X_test = scaler.transform(X_test)  

mlp.fit(X,Y)

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=13, learning_rate='constant',
       learning_rate_init=1e-05, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [13]:
predictions = mlp.predict(np.array(X_test))

In [14]:
print(classification_report(Y_test,predictions))

             precision    recall  f1-score   support

          1       0.65      0.61      0.63       225
          3       0.73      0.76      0.74       413
          4       0.14      0.13      0.14        15
         30       0.05      0.05      0.05        42

avg / total       0.65      0.65      0.65       695



In [15]:
star = classification(1,Y_test,predictions)
qso = classification(3,Y_test,predictions)
gal = classification(4,Y_test,predictions)
bal = classification(30,Y_test,predictions)

In [34]:
File_Name = input("Please Enter File name: ")

Please Enter File name: FullRun1


In [35]:
d = open(TrainingDir+slash+File_Name+".txt", 'w')

In [36]:
t1=["Files used to train: ",np.str(Spectra_Files), "\n"]
sp= "\n"
t2 = ["Files used to test: ",np.str(Spectra_TestFiles), "\n"]
t3 = ["Number of training objects = ",,np.str(len(Spectra_Files)), "\n"]
t4 = ["Number of testing objects = ",,np.str(len(Spectra_TestFiles)), "\n"]
n1 = ["Structure of neural network: ", np.str(hiddenlayer_format),"\n"]
n2  = ["Backpropagation method used: ",np.str(backprop_method), "\n"]
n3  = ["Learning rate: ",np.str(lr), "\n"]
n4  = ["Activation Function: ",np.str(act), "\n"]
r1 = ["Results of Neural Network: ", "\n","\n"]
r2=["        "," Star    Quasar  Galaxy  BAL ","\n",]
r3="Star     ",np.str(np.round(star[0]*100,2)),"%  ", np.str(np.round(star[1]*100,2)),"%  ",np.str(np.round(star[2]*100,2)),"%  ",np.str(np.round(star[3]*100,2)),"%","\n"
r4="Quasar   ",np.str(np.round( qso[0]*100,2)),"%  ", np.str(np.round( qso[1]*100,2)),"%  ",np.str(np.round( qso[2]*100,2)),"%  ",np.str(np.round( qso[3]*100,2)),"%","\n"
r5="Galaxy   ",np.str(np.round( gal[0]*100,2)),"%  ", np.str(np.round( gal[1]*100,2)),"%  ",np.str(np.round( gal[2]*100,2)),"%  ",np.str(np.round( gal[3]*100,2)),"%","\n"
r6="BAL      ",np.str(np.round( bal[0]*100,2)),"%  ", np.str(np.round( bal[1]*100,2)),"%  ",np.str(np.round( bal[2]*100,2)),"%  ",np.str(np.round (bal[3]*100,2)),"%","\n"

In [37]:
d.writelines(t1)
d.writelines(sp)
d.writelines(t2)
d.writelines(sp
d.writelines(t3)
d.writelines(t4)
d.writelines(sp)
d.writelines(sp)
d.writelines(n1)
d.writelines(n2)
d.writelines(n3)
d.writelines(n4)
d.writelines(sp)
d.writelines(sp)
d.writelines(r1)
d.writelines(r2)
d.writelines(r3)
d.writelines(r4)
d.writelines(r5)
d.writelines(r6)


In [38]:
list.close()

that HDU was no read while the HDUList was open
is deprecated.  That is, you did something like:

    >>> hdulist.close()
    >>> print(hdulist[2].header)

even though hdulist[2] had not been read yet.  Instead
do:

    >>> print(hdulist[2].header)
    >>> hdulist.close()

or open the file with lazy_load_hdus=False to read all
the HDUs into memory immediately upon opening the file.
 [astropy.io.fits.hdu.hdulist]


In [39]:
d.close()